In [30]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [31]:
import pickle
train_path = '/content/drive/MyDrive/ssne-recursive/train.pkl'
with open(train_path, 'rb') as f:
    data = pickle.load(f)

`data` jest postaci: `{(array([float, float, ..., float]), int), ...}`, gdzi tablica floatów to utwór, a int to oznaczenie kompozytora

In [34]:
print(f"Length: {len(data)}, Type: {type(data)}")
print(f"Length[0]: {len(data[0])}")
print(data[0])
print(len(data[0][0]))
print(data[0][0][0])

Length: 2939, Type: <class 'list'>
Length[0]: 2
(array([0.        , 0.        , 0.        , ..., 0.41361257, 0.21465969,
       0.7591623 ]), 0)
4756
0.0


In [33]:
import math

min_val = math.inf
max_val = -math.inf

for x in data:
    for elt in x[0]:
        if elt < min_val:
            min_val = elt
        if elt > max_val:
            max_val = elt

for i in range(len(data)):
    for j in range(len(data[i][0])):
        data[i][0][j] = (data[i][0][j] - min_val) / max_val

In [38]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [36]:
train_data = [(torch.tensor(x[0]), x[1]) for x in data]

In [39]:
def collate_fn(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    max_length = max(x_lens)

    padded_xx = [torch.cat([x, torch.zeros(max_length - len(x))]) for x in xx]

    return torch.stack(padded_xx), torch.tensor(yy)

In [40]:
train_loader = DataLoader(dataset=train_data, batch_size=32, collate_fn=collate_fn)

In [18]:
composers = {0: 'bach', 1: 'beethoven', 2: 'debussy', 3: 'scarlatti', 4: 'victoria'}